In [2]:
%matplotlib inline
import pandas as pd 
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.formula.api import ols
from scipy import stats

In [3]:
data = pd.read_csv("playlists.csv", sep=";", encoding = "ISO-8859-1") 

### Negatives and positives

In [4]:
companies = data['company'].unique()
by_company = [data[data.company == company] for company in companies]
positives = []
negatives = []
for data_com in by_company:
    data_com = data_com.sort_values('playlist_sample')
    last_pl = int(data_com.tail(1).playlist_sample)
    #pls = pd.DataFrame({'pl':range (1,last_pl+1), 'old':[0]*last_pl, 'new':[0]*last_pl})
    #curr_pl = data_com.query('playlist_sample == '+str(1))
    #pls.new[0]=(curr_pl.shape[0])/3
    #sum_pls = [curr_pl.shape[0]/3]
    #for i in range(2,last_pl+1):
    #    curr_pl = data_com.query('playlist_sample == '+str(i))
    #    pre_pl = data_com.query('playlist_sample == '+str(i-1))
    #    olds = curr_pl['song'].map(pre_pl['song'].value_counts()).sum(axis = 0)/3
    #    pls.old[i-1]= olds/3 
    #    pls.new[i-1]=(curr_pl.shape[0]-olds)/3
    #    sum_pls.append(curr_pl.shape[0]/3)
    #sum_pls = pd.DataFrame(sum_pls)
    #pls_rel = (pls[['old']].div(sum_pls.values, axis=0)*100).join(pls[['new']].div(sum_pls.values, axis=0)*100)
    #pls[['old','new']].plot(kind='bar', stacked=True, title=data_com.iloc[0,0])
    #for n in pls_rel:
    #    for i, (cs, ab, pc) in enumerate(zip(pls.iloc[:, 1:].cumsum(1)[n].values, pls[n].values, pls_rel[n].values)):
    #        if(pc>0):
    #            plt.text(i, cs - ab/2, str(np.round(pc, 1)) + '%', va='center', ha='center')
    df_last_pl= data_com.query('playlist_sample == '+str(last_pl))
    positives.append(df_last_pl)
    pos_loc = pd.DataFrame({}, columns=data_com.columns)
    rep=0
    for index, row in data_com[data_com.playlist_sample<last_pl].iterrows(): 
        if not ((df_last_pl['artist'] == row['artist']) & (df_last_pl['song'] ==  row['song'])).any() and pos_loc[(pos_loc['artist']== row['artist']) & (pos_loc['song'] ==  row['song'])].shape[0]<3:
            pos_loc= pos_loc.append(row, ignore_index=True)
        else:
            rep+=1
    #n_vs_p = pd.DataFrame({'sam':['pos', 'neg'],'num':[df_last_pl.shape[0]/3,pos_loc.shape[0]/3]})
    # n_vs_p.plot.bar(x='sam', y='num', rot=0, title=data_com.iloc[0,0])
    negatives.append(pos_loc)

Count of positives negatives and total by company.

In [5]:
for i in range(len(companies)):
    print(companies[i])
    print("Negatives: ",negatives[i].shape[0]/3)
    print("Positives: ",positives[i].shape[0]/3)
    print("Total: ", by_company[i].shape[0]/3)

Arte Francés
Negatives:  76.0
Positives:  29.0
Total:  147.0
Club De Banqueros y Empresarios
Negatives:  61.0
Positives:  24.0
Total:  88.0
Gramma
Negatives:  51.0
Positives:  21.0
Total:  103.0
Hotel Marrakech
Negatives:  28.0
Positives:  20.0
Total:  68.0
Specialized
Negatives:  38.0
Positives:  35.0
Total:  103.0
Urban Place
Negatives:  48.0
Positives:  24.0
Total:  85.0


Append the positives and negatives records with a new column "chosen" that takes value of 1 if is chosen.

In [6]:
df_n_ps = pd.DataFrame({}, columns= negatives[0].columns)
for i in range(len(negatives)):
    negatives[i]['chosen']=0
    positives[i]['chosen']=1
    df_n_ps = df_n_ps.append(negatives[i].append(positives[i],sort=False), sort=False)

D:\Usuarios\1144084318\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
df_n_ps

,company,playlist_sample,namesfiles,no,artist,song,sampleratefiles,totalsamplesfiles,durationfiles,bitratefiles,...,chromagramfiles_6,chromagramfiles_7,chromagramfiles_8,chromagramfiles_9,chromagramfiles_10,chromagramfiles_11,chromagramfiles_12,attackslopefiles,attackleapfiles,chosen
0,Arte Francés,1,14 - Sonny Alven - Backseat Lovin' (Cavego Rem...,14,Sonny Alven,Backseat Lovin' (Cavego Remix).mp3 ...,44100,10074238,228.440775,320,...,0.703913,0.134777,0.439297,0.414337,0.759675,0.273383,0.463493,10.299749,0.224341,0.0
1,Arte Francés,1,04 - 11 Acorn Lane - Perfect (Electro Swing Re...,4,11 Acorn Lane,Perfect (Electro Swing Remix).mp3 ...,44100,8932606,202.553425,320,...,0.359089,0.301820,1.000000,0.584436,0.797950,0.513893,0.513712,7.917774,0.928387,0.0
2,Arte Francés,1,05 - Dimie Cat - Glam (Electro-swing Remix).mp...,5,Dimie Cat,Glam (Electro-swing Remix).mp3,44100,8249471,187.062825,320,...,0.031553,0.009838,0.968910,0.227496,0.238726,0.314593,0.005669,15.448735,0.235664,0.0
3,Arte Francés,1,05 - Dimie Cat - Glam (Electro-swing Remix).mp...,5,Dimie Cat,Glam (Electro-swing Remix).mp3,44100,8249471,187.062825,320,...,0.083225,0.057891,0.822857,0.360494,0.394564,0.245561,0.124798,8.269979,0.456702,0.0
4,Arte Francés,1,05 - Dimie Cat - Glam (Electro-swing Remix).mp...,5,Dimie Cat,Glam (Electro-swing Remix).mp3,44100,8249471,187.062825,320,...,0.219241,0.288200,0.774249,0.731843,0.534440,0.494625,0.370216,3.696659,0.270895,0.0
5,Arte Francés,1,06 - Goldfish - We Come Together (Fishybeat Mi...,6,Goldfish,We Come Together (Fishybeat Mix).mp3 ...,44100,16881406,382.798325,320,...,0.123232,0.502177,0.762218,0.059202,0.093636,0.170661,0.111771,19.159306,0.895707,0.0
6,Arte Francés,1,06 - Goldfish - We Come Together (Fishybeat Mi...,6,Goldfish,We Come Together (Fishybeat Mix).mp3 ...,44100,16881406,382.798325,320,...,0.336966,0.239061,0.533042,1.000000,0.550048,0.645518,0.866738,15.714549,0.253784,0.0
7,Arte Francés,1,06 - Goldfish - We Come Together (Fishybeat Mi...,6,Goldfish,We Come Together (Fishybeat Mix).mp3 ...,44100,16881406,382.798325,320,...,0.894295,0.232172,0.997104,0.847663,1.000000,0.991279,0.866532,27.669830,0.365340,0.0
8,Arte Francés,1,04 - 11 Acorn Lane - Perfect (Electro Swing Re...,4,11 Acorn Lane,Perfect (Electro Swing Remix).mp3 ...,44100,8932606,202.553425,320,...,0.601880,0.097947,0.425159,0.513959,0.722148,0.731815,0.977127,8.116122,0.272116,0.0
9,Arte Francés,1,11 - Parov Stelar - The Vamp.mp3 ...,11,Parov Stelar,The Vamp.mp3 ...,44100,14205310,322.115875,320,...,0.094671,0.110672,0.617675,0.413602,0.762493,1.000000,0.302813,6.810961,0.833389,0.0


### Preprocessing

In [8]:
from collections import Counter


from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, silhouette_samples, silhouette_score, calinski_harabaz_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
import math
import seaborn as sns

In [9]:
df_n_ps.bitratefiles = df_n_ps.bitratefiles.astype('float64')
df_n_ps.pitchfiles = df_n_ps.pitchfiles.astype('float64')
df_n_ps.bestkeyfiles = df_n_ps.bestkeyfiles.astype('float64')
df_n_ps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365 entries, 0 to 1781
Data columns (total 65 columns):
company                 1365 non-null object
playlist_sample         1365 non-null object
namesfiles              1365 non-null object
no                      1365 non-null object
artist                  1365 non-null object
song                    1365 non-null object
sampleratefiles         1365 non-null object
totalsamplesfiles       1365 non-null object
durationfiles           1365 non-null float64
bitratefiles            1365 non-null float64
rmsfiles                1365 non-null float64
rmsmedianfiles          1365 non-null float64
lowenergyfiles          1365 non-null float64
ASRfiles                1365 non-null float64
beatspectrumfiles       1365 non-null float64
eventdensityfiles       1365 non-null float64
tempofiles              1365 non-null float64
pulseclarityfiles       1365 non-null float64
zerocrossfiles          1365 non-null float64
rolloffsfiles           136

Vamos a reemplazar los NaN y entonces a normalizar los datos para que todas las variables tengan la misma importancia. Solo vamos a considerar los datos numéricos.

In [10]:
df_n_ps = df_n_ps.fillna(0)
df_n_ps_std = pd.DataFrame(preprocessing.scale(df_n_ps.iloc[:,8:]))
df_n_ps_std.columns=df_n_ps.columns[8:]
df_n_ps_std.mean(axis=0)

durationfiles           2.900407e-16
bitratefiles            1.392618e-15
rmsfiles               -4.084645e-16
rmsmedianfiles          6.929744e-17
lowenergyfiles          4.300996e-16
ASRfiles               -1.906493e-16
beatspectrumfiles      -6.003541e-17
eventdensityfiles      -1.646627e-16
tempofiles             -8.819970e-16
pulseclarityfiles       1.861352e-16
zerocrossfiles          5.221708e-17
rolloffsfiles           2.269247e-16
brightnessfiles        -3.316842e-16
spreadfiles            -4.078138e-16
centroidfiles          -3.516926e-16
kurtosisfiles           1.782050e-16
flatnessfiles          -4.287982e-16
entropyfiles            2.354242e-15
mfccfiles_1             2.543346e-16
mfccfiles_2             2.928061e-17
mfccfiles_3            -1.311934e-16
mfccfiles_4             3.290002e-17
mfccfiles_5             1.526862e-16
mfccfiles_6            -2.673889e-17
mfccfiles_7            -9.109522e-18
mfccfiles_8             9.129856e-18
mfccfiles_9            -6.913477e-19
m

In [11]:
df_n_ps_std.std(axis=0)

durationfiles           1.000367
bitratefiles            1.000367
rmsfiles                1.000367
rmsmedianfiles          1.000367
lowenergyfiles          1.000367
ASRfiles                1.000367
beatspectrumfiles       1.000367
eventdensityfiles       1.000367
tempofiles              1.000367
pulseclarityfiles       1.000367
zerocrossfiles          1.000367
rolloffsfiles           1.000367
brightnessfiles         1.000367
spreadfiles             1.000367
centroidfiles           1.000367
kurtosisfiles           1.000367
flatnessfiles           1.000367
entropyfiles            1.000367
mfccfiles_1             1.000367
mfccfiles_2             1.000367
mfccfiles_3             1.000367
mfccfiles_4             1.000367
mfccfiles_5             1.000367
mfccfiles_6             1.000367
mfccfiles_7             1.000367
mfccfiles_8             1.000367
mfccfiles_9             1.000367
mfccfiles_10            1.000367
mfccfiles_11            1.000367
mfccfiles_12            1.000367
mfccfiles_

Borramos pitch porque todos sus valores son 0.

In [12]:
df_n_ps_std = df_n_ps_std.drop(columns="pitchfiles")

In [13]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.0'

In [14]:
from keras.layers import Input, Flatten, Dense#, Lambda
from keras.models import Model
from keras import layers
from keras import models, optimizers

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV #permite buscar la mejor configuración de parámetros con C-V
from sklearn.metrics import make_scorer # permite crear una clase scorer a partir de una función de score (necesario para el kappa)
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

# MFCC

In [15]:
df_n_ps_std.columns[18:31]

Index(['mfccfiles_1', 'mfccfiles_2', 'mfccfiles_3', 'mfccfiles_4',
       'mfccfiles_5', 'mfccfiles_6', 'mfccfiles_7', 'mfccfiles_8',
       'mfccfiles_9', 'mfccfiles_10', 'mfccfiles_11', 'mfccfiles_12',
       'mfccfiles_13'],
      dtype='object')

In [16]:
df_n_ps_std_mfcc = df_n_ps_std.iloc[:,18:31]
df_n_ps_std_mfcc.columns=df_n_ps_std.columns[18:31]
df_n_ps_std_mfcc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1365 entries, 0 to 1364
Data columns (total 13 columns):
mfccfiles_1     1365 non-null float64
mfccfiles_2     1365 non-null float64
mfccfiles_3     1365 non-null float64
mfccfiles_4     1365 non-null float64
mfccfiles_5     1365 non-null float64
mfccfiles_6     1365 non-null float64
mfccfiles_7     1365 non-null float64
mfccfiles_8     1365 non-null float64
mfccfiles_9     1365 non-null float64
mfccfiles_10    1365 non-null float64
mfccfiles_11    1365 non-null float64
mfccfiles_12    1365 non-null float64
mfccfiles_13    1365 non-null float64
dtypes: float64(13)
memory usage: 138.7 KB


## 1-comp groups

In [95]:
X = df_n_ps_std_mfcc.copy()
print(X.shape)
y = []
for i, row in df_n_ps.iterrows():
    if row['company'] == companies[0]:
        y.append(row['chosen'])
    else:
        y.append(0)
y = np.asarray(y)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y)

mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

activation_vec = ['logistic', 'relu', 'tanh']
max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                          (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


np.random.seed(1234)
parametros = {'activation': activation_vec,
              'max_iter':max_iter_vec,
              'hidden_layer_sizes': hidden_layer_sizes_vec,
              'learning_rate_init': learning_rate_init_vec#,
              #'batch_size': batch_size_vec
              }
scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

n0=X_train.shape[1]
### hidden_layer_sizes
ns = []
for i in range (len(grid.best_params_['hidden_layer_sizes'])):
    ns.append(grid.best_params_['hidden_layer_sizes'][i])

ns.append(1)
lr = grid.best_params_['learning_rate_init']
epochs = grid.best_params_['max_iter']

input_tensor = Input(shape = (n0,))
hidden_outputs = [input_tensor]
if grid.best_params_['activation'] == 'logistic':
    grid.best_params_['activation'] = 'sigmoid'
for i in range (len(ns)-1):
    hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

model = Model([input_tensor], [classification_output])
weights = model.get_weights()

print(model.summary())

model.set_weights(weights)
adam = keras.optimizers.Adam(lr=lr)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
            callbacks=[
                keras.callbacks.ReduceLROnPlateau(
                    monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                )
            ]
         )

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

print("epochs: "+str(len(acc)))

test_loss, test_acc = model.evaluate(X_test, y_test)
print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

y_pred = model.predict(X_test)
print("AUC ROC: ",roc_auc_score(y_test, y_pred))

y_pred = list(map(lambda i: int(i>=0.5), y_pred))
print("Kappa: ",cohen_kappa_score(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

(1365, 13)
(1365,)
Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (10, 10), 'learning_rate_init': 0.001, 'max_iter': 400}, que permiten obtener un Accuracy de 94.13% y un Kappa del 14.97
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                140       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________
None
epochs: 400
342/342 [=========

In [96]:
from IPython.display import display, Markdown, Latex
for company in companies[1:]:
    display(Markdown('## '+company))
    X = df_n_ps_std_mfcc.copy()
    print(X.shape)
    y = []
    for i, row in df_n_ps.iterrows():
        if row['company'] == company:
            y.append(row['chosen'])
        else:
            y.append(0)
    y = np.asarray(y)
    print(y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Club De Banqueros y Empresarios

(1365, 13)
(1365,)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (75) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30,), 'learning_rate_init': 0.007, 'max_iter': 75}, que permiten obtener un Accuracy de 95.70% y un Kappa del 27.24
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 30)                420       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 451
Trainable params: 451
Non-trainable params: 0
_________________________________________________________________
None
epochs: 75
342/342 [==============================] - 0s 47us/step
test loss: 0.25140536593937735, test accuracy: 0.9181286692619324
AUC ROC:  0.8104138851802403
Kappa:  -0.031674208144

## Gramma

...

Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate_init': 0.008, 'max_iter': 50}, que permiten obtener un Accuracy de 92.67% y un Kappa del 2.34
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                140       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
None
epochs: 50
342/342 [==============================] - 0s 118us/step
test loss: 0.26170792008003996, test accuracy: 0.9122806787490845
AUC ROC:  0.7642392861077449
Kappa:  0.0473537604456

## Urban Place

(1365, 13)
(1365,)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (20,), 'learning_rate_init': 0.004, 'max_iter': 100}, que permiten obtener un Accuracy de 94.53% y un Kappa del 2.94
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 20)                280       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 21        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________
None
epochs: 100
342/342 [==============================] - 0s 29us/step
test loss: 0.1636523435338896, test accuracy: 0.9590643048286438
AUC ROC:  0.7763506625891947
Kappa:  0.1201764057331

## 2-comp groups

In [ ]:
from IPython.display import display, Markdown, Latex
import itertools
two_combs = list(itertools.combinations(companies, 2))
for comb in two_combs:
    title = '## '
    for comp in comb[:-1]:
        title+=comp
        title+=' y '
    title+=comb[-1]
    display(Markdown(title))
    X = df_n_ps_std_mfcc.copy()
    print(X.shape)
    y = []
    for i, row in df_n_ps.iterrows():
        if row['company'] in comb:
            y.append(row['chosen'])
        else:
            y.append(0)
    # print(y)
    y = np.asarray(y)
    print(y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios

(1365, 13)
(1365,)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30, 20, 10), 'learning_rate_init': 0.008, 'max_iter': 20}, que permiten obtener un Accuracy de 88.47% y un Kappa del 6.35
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 1,261
Trainable params: 1,261
Non-tr

## Arte Francés y Gramma

...

(1365, 13)
(1365,)
Los parámetros del mejor modelo fueron {'activation': 'logistic', 'hidden_layer_sizes': (30, 20, 10), 'learning_rate_init': 0.005, 'max_iter': 500}, que permiten obtener un Accuracy de 86.02% y un Kappa del 4.70
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 30)                420       
_________________________________________________________________
dense_12 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_13 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 1,261
Traina

## Arte Francés y Urban Place

(1365, 13)
(1365,)
Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate_init': 0.002, 'max_iter': 400}, que permiten obtener un Accuracy de 89.44% y un Kappa del 5.97
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                140       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 11        
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
None
epochs: 400
342/342 [==============================] - 0s 32us/step
test loss: 0.40599260710136237, test accuracy: 0.8625730872154236
AUC ROC:  0.6064911648034619
Kap

## Club De Banqueros y Empresarios y Gramma

(1365, 13)
(1365,)
